# Topic 2 - Knapsack Problem  

Given a set of items, each with an associated weight ($w_i$) and value ($v_i$), the objective is to determine the combination of items to be placed in a knapsack with limited capacity ($W$) in order to maximize the total value of the selected items without exceeding the knapsack's capacity.  

Formally, let ($n$) be the number of items, ($w_i$) the weight of item ($i$), ($v_i$) the value of item ($i$), and ($W$) the capacity of the knapsack. The problem can be described by the following maximization function:  

$\text{Maximize} \sum_{i=1}^{n} v_i \cdot x_i $  

subject to:  

$\sum_{i=1}^{n} w_i \cdot x_i \leq W $  

where ($x_i$) is a binary variable indicating whether item ($i$) is included in the knapsack (1 if included, 0 otherwise).  

The **Knapsack Problem** is known to be **NP-hard**, meaning that no efficient algorithm is currently known to solve it in polynomial time for all cases.

## Dataset Generation

In [1]:
import numpy as np

def create_ksp(number_of_items, max_utility = 100, max_weight = 100):
    utility = np.random.randint(0, max_utility, number_of_items)
    capacity = np.random.randint(0, max_weight, number_of_items)
    
    return utility, capacity
    


## Objective Function

In [3]:
def objective_function(utility, capacity, max_capacity, choice):
    c = np.dot(capacity, choice)
    if c <= max_capacity:
        return np.dot(utility, choice)
    else:
        return 0

## Population Generation
My population are choices based on 1s and 0s so i have to replicate choices given before to build my population

In [4]:
def populacao_inicial(n_pop, n_itens):
    return [np.random.choice([0,1],n_itens) for _ in range(n_pop)]

## Selection Candidates
Made by tournament
Each candidate is actually an representation of 0's and 1's that tells me if an item is chosen to be in the bag or not

In [5]:
def selection(pop, scores, k=3):
    selection_ix = np.random.randint(len(pop))
    for ix in np.random.randint(0, len(pop), k-1):
        if scores[ix] > scores[selection_ix]:
            selection_ix = ix
    return pop[selection_ix]

## Cross Over
Basically implements a crossover based on a 50 % rule. 2 childs, one with 1st half of 1st parent and 2nd half of 2nd parent. Another with 1st half of 2nd parent and 2nd half of 1st parent.

In [6]:
def crossover(p1, p2, r_cross):
    c1, c2 = p1.copy(), p2.copy()
    if np.random.rand() < r_cross:
        pt = int(0.5 * len(p1))
        c1 = np.concatenate((p1[:pt], p2[pt:]))
        c2 = np.concatenate((p2[:pt], p1[pt:]))
    return np.array([c1, c2])

## Mutation

A mutation is when one of the randomly chosen items changes from being selected to not selected or vice-versa

In [7]:
def mutation(chosen, r_mut):
    for i in range(len(chosen)):
        if np.random.rand() < r_mut:
            chosen[i] = int(not bool(chosen[i]))

## Genetic Algorithm
With implementation of previous functions

In [8]:
def algoritmo_genetico(objetive, n_itens, utility, capacity, max_capacity, n_pop, n_gen, p_mut, p_cross):
    
    pop = populacao_inicial(n_pop, n_itens)
        
    best, best_eval = pop[0], objetive(utility,capacity,max_capacity,pop[0])
    
    for gen in range(n_gen):
        scores = [objetive(utility, capacity,max_capacity,c) for c in pop]
        
        for i in range(n_pop):
            if scores[i] > best_eval:
                best, best_eval = pop[i], scores[i]
                print(f'Found new best in generation {gen} with capacity {np.dot(capacity,pop[i])} and utility {scores[i]}')
                
        # Selection
        selected = [selection(pop, scores) for _ in range(n_pop)]
        
        children = list()
        # Reproduction
        
        for i in range(0, n_pop, 2):
            p1, p2 = selected[i], selected[i+1]
            
            for c in crossover(p1 ,p2, p_cross):
                mutation(c,p_mut)
                children.append(c)
        
        pop = children
    
    return [best, best_eval]


## Inputs from user

In [9]:
def get_inputs():
    n_itens = int(input('Number of Items: '))
    max_capacity_item = int(input('Maximum weight of itens: '))
    max_utility_item = int(input('Maximum utility of itens: '))
    max_capacity_bag = int(input('Maximum bag weight: '))
    n_pop = int(input('Population Size: '))
    n_gen = int(input('Number of generations: '))
    mutation_rate = float(input('Mutation Rate: '))
    crossover_rate = float(input('Crossover Rate: '))    
    return n_itens, max_capacity_item, max_utility_item, max_capacity_bag, n_pop, n_gen, mutation_rate, crossover_rate

# Structure of Solution
1. Inputs
2. Generate Items
2. Application of Algorithm

In [10]:
# 1
n_itens, max_capacity, max_utility, max_capacity_bag, n_pop, n_gen, mutation_rate, crossover_rate = get_inputs()

# 2
utility, capacity = create_ksp(n_itens,max_utility,max_capacity)

# 3
result= algoritmo_genetico(objective_function, n_itens, utility, capacity, max_capacity_bag, n_pop, n_gen, mutation_rate, crossover_rate)

Found new best in generation 0 with capacity 2334 and utility 2290
Found new best in generation 0 with capacity 2298 and utility 2364
Found new best in generation 0 with capacity 2227 and utility 2377
Found new best in generation 0 with capacity 2413 and utility 2499
Found new best in generation 0 with capacity 2486 and utility 2559
Found new best in generation 0 with capacity 2179 and utility 2630
Found new best in generation 0 with capacity 2249 and utility 2699
Found new best in generation 0 with capacity 2346 and utility 2890
Found new best in generation 0 with capacity 2478 and utility 3002
Found new best in generation 3 with capacity 2459 and utility 3032
Found new best in generation 4 with capacity 2478 and utility 3044
Found new best in generation 4 with capacity 2499 and utility 3063
Found new best in generation 5 with capacity 2448 and utility 3127
Found new best in generation 6 with capacity 2491 and utility 3149
Found new best in generation 6 with capacity 2431 and utility 

In [11]:
print(f'Best solution for {n_gen} generations has capacity {np.dot(capacity,result[0])} and utility {result[1]}')

Best solution for 200 generations has capacity 2464 and utility 3675
